In [1]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import time
import dill

import sys
sys.path += ['Package/']  ### Specify the directory where the Package is
from functions_for_DDCSBM import *
from dynamic_SC_BH import *

import warnings
warnings.filterwarnings("ignore")

In [ ]:
N_av = 40 # number of experiments for averaging

## Computation times of Algo 2

Here we look at the behavior of the computation time of Algo 2 wrt the number of nodes n, the number of timesteps T, and the number of classes k. Experiments are done for a fixed $\alpha=1.5 \alpha_c$, a fixed $\eta=0.7$, a fixed average degree $c=6$ and a fixed $\phi=1.6$. 

The following cell runs in a bit more than 2 hours on a laptop

In [10]:
nn = [500, 1000, 1500, 2000]
TT = [2, 5, 10, 15]
N_CLUST = [2, 5, 10, 15] # number of communities

#c_out = 3.5 # average number of connections between nodes in different communities
alpha_norm = 1.5
c = 6 # average degree
eta = 0.7 # label persistence
phi = 1.6

######

COMP_TIME_EIG_JL = np.zeros((len(nn), len(TT), len(N_CLUST), N_av))
COMP_TIME_KMEANS_JL = np.zeros((len(nn), len(TT), len(N_CLUST), N_av))

######

for i in np.arange(len(nn)):
    n=nn[i]
    for j in np.arange(len(TT)):
        T=TT[j]
        alpha_c = find_transition(T,eta) # critical value of the transition
        alpha = alpha_norm * alpha_c
        lambd = alpha / np.sqrt(c*phi)
        c_out = c*(1-lambd)
        for k in np.arange(len(N_CLUST)):
            n_clusters = N_CLUST[k]
            for l in np.arange(N_av):
                print([n, T, n_clusters,l])
                theta = np.random.uniform(3,10,n)**3 # vector theta
                theta = theta/np.mean(theta) # E[theta_i] = 1
                fluctuation = 0/n_clusters # fluctuations of off-diagonal elements of C (see documentation)
                fraction = np.random.normal(1/n_clusters, 0/(4*n_clusters),n_clusters) # vector pi
                fraction = fraction/np.sum(fraction) # \sum_i \pi_i = 1
                C = matrix_C(c_out, c,fluctuation, fraction) # affinity matrix C
                AT, Label = adjacency_matrix_series_DDCSBM(T, C, c, eta, theta,fraction) 
                cluster_JL = dynamic_community_detection_JL(AT, eta, real_classes = Label, n_clusters = n_clusters)
                COMP_TIME_EIG_JL[i,j,k,l] = cluster_JL.eigspace_time
                COMP_TIME_KMEANS_JL[i,j,k,l] = cluster_JL.kmeans_time
                
dill.dump_session('Alg_2_computation_time_vs_T_n_and_k_alpha_norm=1p5.db') #save for further use (draw figures)

## Compare Algos 1 and 2 -- accuracy versus $\alpha$

Here we compare the overlap performance of Algo 2 versus that of Algo 1, wrt to the control parameter $\alpha$. Experiments are done for a fixed number of nodes $n=5000$, a fixed number of timesteps $T=10$, a fixed number of clusters $k=2$, a fixed $\eta=0.5$, a fixed average degree $c=6$ and a fixed $\phi=1.6$. 

The following cell runs in a bit more than 2 hours on a laptop

In [12]:
n = 5000
T = 10
n_clusters = 2
ALPHA_NORM = np.linspace(0.5, 2.5, 9)

c = 6 # average degree
eta = 0.5 # label persistence

alpha_c = find_transition(T,eta) # critical value of the transition
phi = 1.6 # phi

ALPHA = ALPHA_NORM * alpha_c
LAMBD = ALPHA / np.sqrt(c*phi)
C_OUT = c*(1-LAMBD)

######

AV_OVERLAP_JL = np.zeros((len(C_OUT), N_av))
AV_OVERLAP = np.zeros((len(C_OUT), N_av))

######

for j in np.arange(len(C_OUT)):
    c_out = C_OUT[j]
    for k in np.arange(N_av):
        print([c_out, k])
        theta = np.random.uniform(3,10,n)**3 # vector theta
        theta = theta/np.mean(theta) # E[theta_i] = 1
        fluctuation = 0/n_clusters # fluctuations of off-diagonal elements of C (see documentation)
        fraction = np.random.normal(1/n_clusters, 0/(4*n_clusters),n_clusters) # vector pi
        fraction = fraction/np.sum(fraction) # \sum_i \pi_i = 1
        C = matrix_C(c_out, c, fluctuation, fraction) # affinity matrix C
        AT, Label = adjacency_matrix_series_DDCSBM(T, C, c, eta, theta,fraction) 
        cluster_JL = dynamic_community_detection_JL(AT, eta, real_classes = Label, n_clusters = n_clusters)
        AV_OVERLAP_JL[j,k] = np.mean(cluster_JL.overlap)

        cluster = dynamic_community_detection(AT, eta, real_classes = Label, n_clusters = n_clusters)
        AV_OVERLAP[j,k] = np.mean(cluster.overlap)
        
dill.dump_session('compare_Algs1and2_accuracy_n=5000_T=10_k=2_vs_alpha.db') #save for further use (draw figures)

## Compare Algos 1 and 2 -- computation time

Here we compare the computation time of Algo 2 versus that of Algo 1, wrt to the the number of timesteps $T$.  Experiments are done for a fixed number of nodes $n=300$, a fixed number of clusters $k=2$, a fixed $\eta=0.5$, a fixed average degree $c=6$, a fixed $\phi=1.6$, and a fixed $\alpha=1.5 \alpha_c$.

The following cell runs in around 3.5 hours on a laptop

In [15]:
n = 300
TT = [10, 30, 50, 70, 90, 110]
n_clusters = 2
alpha_norm = 1.5

c = 6 # average degree
eta = 0.5 # label persistence
phi = 1.6 # phi

######

COMP_TIME_EIG_JL = np.zeros((len(TT), N_av))
COMP_TIME_KMEANS_JL = np.zeros((len(TT), N_av))

COMP_TIME_EIG = np.zeros((len(TT), N_av))
COMP_TIME_KMEANS = np.zeros((len(TT), N_av))

######

for j in np.arange(len(TT)):
    T = TT[j]
    alpha_c = find_transition(T,eta) # critical value of the transition
    alpha = alpha_norm * alpha_c
    lambd = alpha / np.sqrt(c*phi)
    c_out = c*(1-lambd)
    for k in np.arange(N_av):
        print([T, k])
        theta = np.random.uniform(3,10,n)**3 # vector theta
        theta = theta/np.mean(theta) # E[theta_i] = 1
        fluctuation = 0/n_clusters # fluctuations of off-diagonal elements of C (see documentation)
        fraction = np.random.normal(1/n_clusters, 0/(4*n_clusters),n_clusters) # vector pi
        fraction = fraction/np.sum(fraction) # \sum_i \pi_i = 1
        C = matrix_C(c_out, c, fluctuation, fraction) # affinity matrix C
        AT, Label = adjacency_matrix_series_DDCSBM(T, C, c, eta, theta,fraction) 
        cluster_JL = dynamic_community_detection_JL(AT, eta, real_classes = Label, n_clusters = n_clusters)
        COMP_TIME_EIG_JL[j,k] = cluster_JL.eigspace_time
        COMP_TIME_KMEANS_JL[j,k] = cluster_JL.kmeans_time

        cluster = dynamic_community_detection(AT, eta, real_classes = Label, n_clusters = n_clusters)
        COMP_TIME_EIG[j,k] = cluster.eigspace_time
        COMP_TIME_KMEANS[j,k] = cluster.kmeans_time
        
dill.dump_session('compare_Algs1and2_comp_time_n=300_k=2_alpha_norm=1p5_vs_T.db') #save for further use (draw figures)

[10, 0]
[10, 1]
[10, 2]
[10, 3]
[10, 4]
[10, 5]
[10, 6]
[10, 7]
[10, 8]
[10, 9]
[10, 10]
[10, 11]
[10, 12]
[10, 13]
[10, 14]
[10, 15]
[10, 16]
[10, 17]
[10, 18]
[10, 19]
[10, 20]
[10, 21]
[10, 22]
[10, 23]
[10, 24]
[10, 25]
[10, 26]
[10, 27]
[10, 28]
[10, 29]
[10, 30]
[10, 31]
[10, 32]
[10, 33]
[10, 34]
[10, 35]
[10, 36]
[10, 37]
[10, 38]
[10, 39]
[30, 0]
[30, 1]
[30, 2]
[30, 3]
[30, 4]
[30, 5]
[30, 6]
[30, 7]
[30, 8]
[30, 9]
[30, 10]
[30, 11]
[30, 12]
[30, 13]
[30, 14]
[30, 15]
[30, 16]
[30, 17]
[30, 18]
[30, 19]
[30, 20]
[30, 21]
[30, 22]
[30, 23]
[30, 24]
[30, 25]
[30, 26]
[30, 27]
[30, 28]
[30, 29]
[30, 30]
[30, 31]
[30, 32]
[30, 33]
[30, 34]
[30, 35]
[30, 36]
[30, 37]
[30, 38]
[30, 39]
[50, 0]
[50, 1]
[50, 2]
[50, 3]
[50, 4]
[50, 5]
[50, 6]
[50, 7]
[50, 8]
[50, 9]
[50, 10]
[50, 11]
[50, 12]
[50, 13]
[50, 14]
[50, 15]
[50, 16]
[50, 17]
[50, 18]
[50, 19]
[50, 20]
[50, 21]
[50, 22]
[50, 23]
[50, 24]
[50, 25]
[50, 26]
[50, 27]
[50, 28]
[50, 29]
[50, 30]
[50, 31]
[50, 32]
[50, 33]
[50,

## Draw figures

In [3]:
dill.load_session('Alg_2_computation_time_vs_T_n_and_k_alpha_norm=1p5.db')
COMP_TIME_TOT_JL = COMP_TIME_KMEANS_JL + COMP_TIME_EIG_JL

In [4]:
fig = plt.figure(figsize = (15,12))

ax1 = plt.subplot2grid((15,17), (5,0), colspan=5, rowspan=5)
ax2 = plt.subplot2grid((15,17), (5,6), colspan=5, rowspan=5)
ax3 = plt.subplot2grid((15,17), (5,12), colspan=5, rowspan=5)

fs = 20
lfs = 15
ts = 15
ax1.set_ylabel('computation time (sec)',fontsize=fs)
ax1.set_xlabel('n',fontsize=fs)
ax1.tick_params(axis="x", labelsize=ts)
ax1.tick_params(axis="y", labelsize=ts)
#ax1.tick_params(top='off', bottom='off', left='off', right='off', labelleft='off', labelbottom='on')
C_vs_n = np.mean(np.squeeze(COMP_TIME_TOT_JL[:,0,:,:]), axis=2)
ax1.plot(nn, C_vs_n[:,0], marker='o', linewidth = 1.4, label='k=2')
ax1.plot(nn, C_vs_n[:,1], marker='s', linewidth = 1.4, linestyle='dashed', label='k=5')
ax1.plot(nn, C_vs_n[:,2], marker='<', linewidth = 1.4, linestyle='dotted', label='k=10')
ax1.plot(nn, C_vs_n[:,3], marker='d', linewidth = 1.4, linestyle='dashdot', label='k=15')
ax1.legend()
ax1.legend(fontsize = lfs)

C_vs_T = np.mean(np.squeeze(COMP_TIME_TOT_JL[1,:,:,:]), axis=2)
ax2.set_xlabel('T',fontsize=fs)
ax2.tick_params(axis="x", labelsize=ts)
ax2.tick_params(axis="y", labelsize=ts)
#ax2.tick_params(top='off', bottom='off', left='off', right='off', labelleft='off', labelbottom='on')
ax2.plot(TT, C_vs_T[:,0], marker='o', linewidth = 1.4, label='k=2')
ax2.plot(TT, C_vs_T[:,1], marker='s', linewidth = 1.4, linestyle='dashed', label='k=5')
ax2.plot(TT, C_vs_T[:,2], marker='<', linewidth = 1.4, linestyle='dotted', label='k=10')
ax2.plot(TT, C_vs_T[:,3], marker='d', linewidth = 1.4, linestyle='dashdot', label='k=15')
ax2.legend()
ax2.legend(fontsize = lfs)

C_vs_k = np.transpose(np.mean(np.squeeze(COMP_TIME_TOT_JL[1,:,:,:]), axis=2))
ax3.set_xlabel('k',fontsize=fs)
ax3.tick_params(axis="x", labelsize=ts)
ax3.tick_params(axis="y", labelsize=ts)
#ax3.tick_params(top='off', bottom='off', left='off', right='off', labelleft='off', labelbottom='on')
ax3.plot(N_CLUST, C_vs_k[:,0], marker='o', linewidth = 1.4, label='T=2')
ax3.plot(N_CLUST, C_vs_k[:,1], marker='s', linewidth = 1.4, linestyle='dashed', label='T=5')
ax3.plot(N_CLUST, C_vs_k[:,2], marker='<', linewidth = 1.4, linestyle='dotted', label='T=10')
ax3.plot(N_CLUST, C_vs_k[:,3], marker='d', linewidth = 1.4, linestyle='dashdot', label='T=15')
ax3.legend()
ax3.legend(fontsize = lfs)

fig.savefig('Algo2_time_vs_n_T_k.pdf', format='pdf', dpi=1200, bbox_inches = 'tight')

In [5]:
dill.load_session('compare_Algs1and2_accuracy_n=5000_T=10_k=2_vs_alpha.db')
dill.load_session('compare_Algs1and2_comp_time_n=300_k=2_alpha_norm=1p5_vs_T.db')

In [6]:
COMP_TIME_TOT_JL = COMP_TIME_KMEANS_JL + COMP_TIME_EIG_JL
COMP_TIME_TOT = COMP_TIME_KMEANS + COMP_TIME_EIG

In [8]:
fig = plt.figure(figsize = (15,12))

ax1 = plt.subplot2grid((15,11), (5,0), colspan=5, rowspan=5)
ax2 = plt.subplot2grid((15,11), (5,6), colspan=5, rowspan=5)

fs = 20
lfs = 15
ts = 15
ax1.set_ylabel('overlap',fontsize=fs)
ax1.set_xlabel(r'$\alpha/\alpha_c(T,\eta)$',fontsize=fs)
ax1.tick_params(axis="x", labelsize=ts)
ax1.tick_params(axis="y", labelsize=ts)
#ax1.tick_params(top='off', bottom='off', left='off', right='off', labelleft='off', labelbottom='on')
ax1.plot(ALPHA_NORM, np.mean(AV_OVERLAP, axis=1), marker='o', linewidth = 1.4, label='Agorithm 1')
ax1.plot(ALPHA_NORM, np.mean(AV_OVERLAP_JL, axis=1), marker='s', linewidth = 1.4, linestyle='dashed', label='Algorithm 2')
ax1.legend()
ax1.legend(fontsize = lfs)

ax2.set_ylabel('computation time (sec)',fontsize=fs)
ax2.set_xlabel('T',fontsize=fs)
ax2.tick_params(axis="x", labelsize=ts)
ax2.tick_params(axis="y", labelsize=ts)
#ax2.tick_params(top='off', bottom='off', left='off', right='off', labelleft='off', labelbottom='on')
ax2.plot(TT, np.mean(COMP_TIME_TOT, axis=1), marker='o', linewidth = 1.4, label='Agorithm 1')
ax2.plot(TT, np.mean(COMP_TIME_TOT_JL, axis=1), marker='s', linewidth = 1.4, linestyle='dashed', label='Agorithm 2')
ax2.legend()
ax2.legend(fontsize = lfs)

fig.savefig('Algo2_vs_Algo1.pdf', format='pdf', dpi=1200, bbox_inches = 'tight')